In [1]:
# Import the CARLA Python API library and some utils
import sys

try:
    sys.path.append('/home/karenli/carla/PythonAPI/carla/dist/carla-0.9.14-py3.8-linux-x86_64.egg')
except IndexError:
    print('Error: CARLA PythonAPI not found.')
    pass

import carla 
import random 

In [2]:
# Connect to the client and get the world object
client = carla.Client('localhost', 2000)
client.load_world('Town07')
world = client.get_world() 

In [3]:
# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [4]:
# Get the blueprint for the vehicle you want
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [5]:
# Move the spectator behind the vehicle 
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

In [6]:
# Add traffic to the simulation
for i in range(30): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 

In [7]:
# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 

In [8]:
# Spawn an RGB cammera with an offset from the vehicle center
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [11]:
# Start the camera saving data to disk
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [12]:
# Stop the camera when we've recorded enough data
camera.stop()

: 